In [16]:
import minsearch

In [17]:
import json

In [18]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [19]:
#print(docs_raw[0:1])

In [20]:
for course_dict in docs_raw:
    print(course_dict['course'])

data-engineering-zoomcamp
machine-learning-zoomcamp
mlops-zoomcamp


In [21]:
#transform the json into "dictionnaries"
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [22]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [23]:
#create index - we specify which words are text field vs key word field
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

#select * where course = 'data-engineering-zoomcamp'

In [24]:
q = 'the course has already started, can I still enroll?'

In [25]:
index.fit(documents)

In [26]:
#implement boosting
#if the question field contains word course than if it is more important than if the answer contains this word
def search(query):
    boost = {'question': 3.0, 'section': 0.5} #default importance is one, we can give less importance by giving 0.5 ore more by giving 3

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [27]:
search(q)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [ ]:
import openai
from openai import OpenAI
client = OpenAI(api_key = OPENAI_API_KEY)

In [29]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [30]:
llm('How do I go to Canada from Chicago?')

'There are several ways to travel from Chicago to Canada, depending on your destination in Canada and your preferences for time, budget, and convenience. Here are some common options:\n\n### By Air:\n1. **Direct Flights:**\n   - Locate a flight from Chicago O\'Hare International Airport (ORD) or Chicago Midway International Airport (MDW) to major Canadian cities like Toronto, Montreal, Vancouver, or Ottawa. \n   - Airlines such as Air Canada, United Airlines, and American Airlines offer direct flights.\n\n2. **Connecting Flights:**\n   - If your destination is not a major city, you might need a connecting flight via hubs like Toronto Pearson International Airport.\n\n### By Train:\n- Amtrak offers train service from Chicago to Toronto via the "Wolverine" to "Maple Leaf" route. This journey requires a transfer in Buffalo, NY. Expect a long travel time, but enjoy scenic views along the way.\n\n### By Bus:\n- Companies such as Greyhound or Megabus provide bus services from Chicago to vari

In [31]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [32]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [33]:
rag(q)

"Yes, you can still enroll in the course after it has started. You are eligible to submit the homework assignments, but please be mindful of the deadlines for turning in final projects. It's important not to leave everything until the last minute."

In [34]:
search(q)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [35]:
from elasticsearch import Elasticsearch

In [36]:
es_client = Elasticsearch('http://localhost:9200') 

In [37]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/-_AUmqKBTTe_nMiiVvcrpA] already exists')

In [38]:
#progress bar
from tqdm.auto import tqdm

In [39]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████| 948/948 [00:02<00:00, 352.64it/s]


In [40]:
query = 'I just disovered the course. Can I still join it?'

In [41]:
def elastic_search(query):
    search_query = {
        "size": 5, #only 5 results or answers
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], #^3 means question is 3x more important than text or section
                        "type": "best_fields"
                    }
                },
                "filter": { #basic filtering like in minsearch
                    "term": {
                        "course": "data-engineering-zoomcamp" 
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [42]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [43]:
rag(query)

'Yes, you can still join the course even after it has started. You are eligible to submit the homeworks, but keep in mind that there are deadlines for turning in the final projects. So, it is advisable not to leave everything for the last minute.'